## Create the required HDFS tables in your user space

#### The following prepares your personal environment and only has to be done once!

In [ ]:
echo "You are ${HADOOP_USER_NAME:-nobody}"
echo "You are ${JUPYTERHUB_USER:-nobody}"

In [ ]:
beeline -n "${JUPYTERHUB_USER:-nobody}" -p "" -e "show databases like '${JUPYTERHUB_USER:-nobody}'"

#### 1. Checkout the content of your database:

In [ ]:
beeline -n "${JUPYTERHUB_USER:-nobody}" -p "" -e "show tables in ${JUPYTERHUB_USER:-nobody};"

#### 2. If you have any databases and you are ok losing their content, drop your database then reconstruct it:

In [ ]:
# beeline -n "${JUPYTERHUB_USER:-nobody}" -p "" -e "drop database if exists ${JUPYTERHUB_USER:-nobody} cascade;"

In [ ]:
beeline -n "${JUPYTERHUB_USER:-nobody}" -p "" -e "show databases like '${JUPYTERHUB_USER:-nobody}';"

Reconstruct your hive folder and reconfigure ACL permissions on HDFS:

In [ ]:
hdfs dfs -ls /user/${JUPYTERHUB_USER:-nobody}

In [ ]:
hdfs dfs -rm -r -f -skipTrash /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -rm -r -f -skipTrash /user/${usJUPYTERHUB_USER:-nobody}/.Trash

In [ ]:
hdfs dfs -mkdir -p                                /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -setfacl    -m group::r-x                /user/${JUPYTERHUB_USER:-nobody}
hdfs dfs -setfacl    -m other::---                /user/${JUPYTERHUB_USER:-nobody}
hdfs dfs -setfacl    -m default:group::r-x        /user/${JUPYTERHUB_USER:-nobody}
hdfs dfs -setfacl    -m default:other::---        /user/${JUPYTERHUB_USER:-nobody}
hdfs dfs -setfacl -R -m group::r-x                /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -setfacl -R -m other::---                /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -setfacl -R -m default:group::r-x        /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -setfacl -R -m default:other::---        /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -setfacl    -m user:hive:rwx             /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -setfacl    -m default:user:hive:rwx     /user/${JUPYTERHUB_USER:-nobody}/hive

#### Creates the main SBB table in your personal HDFS:

In [ ]:
beeline -n "${JUPYTERHUB_USER:-nobody}" -p "" -e "
create database if not exists ${JUPYTERHUB_USER:-nobody} location '/user/${JUPYTERHUB_USER:-nobody}/hive';
drop table if exists ${JUPYTERHUB_USER:-nobody}.sbb_orc;
create external table ${JUPYTERHUB_USER:-nobody}.sbb_orc(
        BETRIEBSTAG string,
        FAHRT_BEZEICHNER string,
        BETREIBER_ID string,
        BETREIBER_ABK string,
        BETREIBER_NAME string,
        PRODUKT_ID string,
        LINIEN_ID string,
        LINIEN_TEXT string,
        UMLAUF_ID string,
        VERKEHRSMITTEL_TEXT string,
        ZUSATZFAHRT_TF string,
        FAELLT_AUS_TF string,
        BPUIC string,
        HALTESTELLEN_NAME string,
        ANKUNFTSZEIT string,
        AN_PROGNOSE string,
        AN_PROGNOSE_STATUS string,
        ABFAHRTSZEIT string,
        AB_PROGNOSE string,
        AB_PROGNOSE_STATUS string,
        DURCHFAHRT_TF string
    )
    row format delimited fields terminated by ';'
    stored as ORC
    location '/data/sbb/orc/istdaten'
    tblproperties ('skip.header.line.count'='1');
    select count(*) from ${JUPYTERHUB_USER:-nobody}.sbb_orc;
    "

#### Creates the a table for SBB geostops in your personal HDFS:

In [ ]:
beeline -n "${JUPYTERHUB_USER:-nobody}" -p "" -e "
create database if not exists ${JUPYTERHUB_USER:-nobody} location '/user/${JUPYTERHUB_USER:-nobody}/hive';
drop table if exists ${JUPYTERHUB_USER:-nobody}.sbb_geostops;
create external table ${JUPYTERHUB_USER:-nobody}.sbb_geostops(
        STATIONID string,
        REMARK string,
        LATITUDE string,
        LONGITUDE string,
        HEIGHT string
    )
    row format delimited fields terminated by ';'
    stored as ORC
    location '/data/sbb/orc/geostops'
    tblproperties ('skip.header.line.count'='1');
    select count(*) from ${JUPYTERHUB_USER:-nobody}.sbb_geostops;
    "

#### Check the tables have been correctly created:

In [ ]:
beeline -n "${JUPYTERHUB_USER:-nobody}" -p "" -e "show tables in ${JUPYTERHUB_USER:-nobody};"